<a href="https://colab.research.google.com/github/linbinbin/Langchain_practice/blob/main/colab/docx_Gragh_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- このノートブックの内容は2024年5月時点の [LangChain 公式ドキュメント](https://python.langchain.com/v0.1/docs/get_started/introduction)に依拠しています。
- 事前に社内規定PDFを概要欄リンクからダウンロードし、動画内の説明に従ってアップロードしておいてください。

## 準備：API キーの設定

In [ ]:
!pip install -q openai
!pip install -q pypdf
!pip install -q langchain
!pip install -q langchain-openai
!pip install -q langchain-chroma
!pip install -q docx2txt
!pip install -q langchain_community
!pip install -q unstructured
!pip install -q langchain-experimental
!pip install -q neo4j
!pip install -q tiktoken
!pip install -q yfiles_jupyter_graphs

In [ ]:
import os
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough, ConfigurableField
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from google.colab import userdata
import openai
#from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.docstore.document import Document
#loader = Docx2txtLoader("example.docx")
files = ["週報_（5月27日-31日_石岡）_IB3内製支援AA.docx", "週報_（6月3日-7日_石岡）_IB3内製支援AA.docx"]

try:
    import google.colab
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

In [ ]:
# OPENAI_API_KEY を取得
openai.api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai.api_key
os.environ['LANGCHAIN_PROJECT'] = "docx_RAG"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ["NEO4J_URI"] = userdata.get('NEO4J_URI')
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = userdata.get('NEO4J_PSWD')

graph = Neo4jGraph()

# RAG（LLM の知識拡張）
RAG を活用することで、LLM は学習データに含まれない知識を得ることができます。

参照用データとしては、現在 LangChain は以下に対応しています：
- .txt
- .csv
- .html
- .json
- .md
- .pdf
- MS Office ファイル

今回は PDF からの知識抽出を試してみましょう。

In [ ]:

# 各ファイルをロード
loaders = [Docx2txtLoader(file) for file in files]

# 各ファイルからデータをロード
raw_data = []
for loader in loaders:
    for result in loader.load():
        if isinstance(result, Document):
            # loader.load()がDocumentオブジェクトを返した場合
            raw_data.append(result)
        elif isinstance(result, tuple):
            # loader.load()がタプルを返した場合
            content, metadata = result
            raw_data.append(Document(page_content=content, metadata=metadata))
        else:
            # その他の場合はエラーを出力する
            print(f"Unsupported result type from loader: {type(result)}")

# テキストスプリッターの初期化
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=125)

# 各ドキュメントを分割
data = [text_splitter.split_documents([doc]) for doc in raw_data]  # Pass a list of Document objects to split_documents


In [ ]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm_transformer = LLMGraphTransformer(llm=llm)
#graph_documents = llm_transformer.convert_to_graph_documents(data)
# Convert the list of lists into a flat list
flat_data = [item for sublist in data for item in sublist]

# Convert the documents to graph documents
graph_documents = llm_transformer.convert_to_graph_documents(flat_data)


In [ ]:
# 保存された出力を取得
graph = Neo4jGraph()
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
#graph_documents
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!START]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget
showGraph()

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [ ]:
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain.invoke({"question": "Amoがテスト結果を確認中"}).names

['Amo']

In [ ]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:20})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 1000
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
print(structured_retriever("Amoのタスクを知りたい"))

In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
    {structured_data}
    Unstructured data:
    {"#Document ". join(unstructured_data)}
    """
    # print(final_data)
    return final_data

_search_query = RunnableLambda(lambda x: x["question"])

template = """あなたは優秀なAIです。下記のコンテキストを利用してユーザーの質問に丁寧に答えてください。
必ず文脈からわかる情報のみを使用して回答を生成してください。
{context}

ユーザーの質問: {question}"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "[GASより追加で質問あり。明日の返答とさせていただく]どこのタスク？"})

Search query: [GASより追加で質問あり。明日の返答とさせていただく]どこのタスク？


'回答: タスクはGasからの追加質問に関連しており、Gasのタスクになります。Gasは明日の返答を予定しています。'

In [ ]:
chain.invoke({"question": "石岡関連のタスクを出力して"})

Search query: 石岡関連のタスクを出力して


'石岡関連のタスクを出力します。\n\n1. Sdr - CREATOR -> 石岡雅紀\n2. Vda - CREATOR -> 石岡雅紀\n3. Mac - CREATOR -> 石岡雅紀\n4. Sdr、Vda、Pspm, Dtc PriorityのAa担当 - RESPONSIBLE_FOR -> Sdr、Vda、Pspm, Dtc Priority\n5. Gas展開、Adt Nissan Team週次会議 - SUBSEQUENT_TO -> Rhel7.9へアップデート案内にPspmが対象リストに含まれていなかった理由はAmoで問い合わせ中'

## 文埋め込みと類似文書検索

In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
from langchain_chroma import Chroma

In [ ]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

In [ ]:
# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":1})

In [ ]:
query = "VESPA お問合せの内容は？"

retriever.invoke(query)

[Document(page_content='進 捗 状 況 報 告 書（週報）\n\nシステム名\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSDR、VDA、MAC\n\n\n\n2024年6月11\n\n日\n\n報 告 期 間\n\n2024年6月03日 ～ 2024年6月07\n\n日\n\n作 成 者\n\n石岡雅紀\n\n当\n\n\n\n期\n\n\n\n作\n\n\n\n業\n\n\n\n実\n\n\n\n施\n\n工程\n\nSDR、VDA、PSPM, DTC Priority\n\n作業項目\n\nSDR、VDA、PSPM, DTC PriorityのAA担当\n\n\n\n赤字：待ち\n\n緑字：対応予定\n\n---------------------------------------------------------------------------------------------------\n\n□\u3000業務部署（GAS／XX6等）問合せ／周知\u3000\u3000\n\n\u3000\u3000□05/28) OperationStaffMeeting（6件）\n\n\u3000\u3000\u3000\u3000\u3000⇒■05/31) GAS展開、ADT Nissan team週次会議\u3000\n\n\u3000\u3000\u3000\u3000\u3000※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中\n\n\u3000\u3000\u300006/04) OperationStaffMeeting（1件）\n\n\u3000\u3000\u3000\u3000\u3000⇒\u3000AMO確認待ち \n\n\u3000\u3000□GAS問合せ（依頼：証明書期限一覧の作成）\n\n\u3000\u3000\u3000\u3000⇒\u3000■04/01) 松ケ野さん、正英さんレビュー\u3000内部向けの資料としては完了\n\n\u3000\u3000\u3000\u3000⇒\u3000GAS展開用の資料作成中\n\n\u3000\u3000□VESPA問合せ（依頼：VESPAの認証方式変更\u3000IDP⇒

## Retrieval を活用したチャットモデルの制御

### まずはただ聞いてみる

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
gpt4o = ChatOpenAI(model="gpt-4o", temperature=0.1)
out = gpt4o.invoke("VESPAお問合せの内容は？")
print(out.content)

「VESPAお問合せの内容は？」という質問に対して、具体的な情報が不足しているため、一般的な回答を提供します。VESPAに関するお問い合わせ内容は多岐にわたる可能性があります。以下は、一般的な問い合わせ内容の例です：

1. **製品情報**:
   - 新しいモデルの詳細
   - 技術仕様
   - カラーバリエーション
   - 価格

2. **購入に関する質問**:
   - 購入方法
   - 支払いオプション
   - ディーラーの所在地
   - 試乗の予約

3. **アフターサービス**:
   - 保証期間と内容
   - メンテナンスサービス
   - 修理の予約
   - 部品の取り寄せ

4. **カスタマイズ**:
   - アクセサリーのオプション
   - カスタムペイントやデザイン

5. **技術サポート**:
   - トラブルシューティング
   - 使用方法のガイド
   - ソフトウェアアップデート（電動モデルの場合）

6. **その他**:
   - イベント情報
   - プロモーションやキャンペーン
   - 環境への取り組み

具体的な問い合わせ内容に応じて、VESPAの公式ウェブサイトやカスタマーサポートに連絡することをお勧めします。


### 準備：プロンプトテンプレートを作る

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="以下を参照して、質問に答えてください。\n\n{context}\n\n質問：{question}"
)

In [ ]:
example = {"question":"これは質問です", "context":"これは外部知識です"}

prompt_template.invoke(example)

StringPromptValue(text='以下を参照して、質問に答えてください。\n\nこれは外部知識です\n\n質問：これは質問です')

### 準備：抽出器を作る

In [ ]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
retriever.invoke("VESPA お問合せの内容は？")

[Document(page_content='進 捗 状 況 報 告 書（週報）\n\nシステム名\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSDR、VDA、MAC\n\n\n\n2024年6月11\n\n日\n\n報 告 期 間\n\n2024年6月03日 ～ 2024年6月07\n\n日\n\n作 成 者\n\n石岡雅紀\n\n当\n\n\n\n期\n\n\n\n作\n\n\n\n業\n\n\n\n実\n\n\n\n施\n\n工程\n\nSDR、VDA、PSPM, DTC Priority\n\n作業項目\n\nSDR、VDA、PSPM, DTC PriorityのAA担当\n\n\n\n赤字：待ち\n\n緑字：対応予定\n\n---------------------------------------------------------------------------------------------------\n\n□\u3000業務部署（GAS／XX6等）問合せ／周知\u3000\u3000\n\n\u3000\u3000□05/28) OperationStaffMeeting（6件）\n\n\u3000\u3000\u3000\u3000\u3000⇒■05/31) GAS展開、ADT Nissan team週次会議\u3000\n\n\u3000\u3000\u3000\u3000\u3000※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中\n\n\u3000\u3000\u300006/04) OperationStaffMeeting（1件）\n\n\u3000\u3000\u3000\u3000\u3000⇒\u3000AMO確認待ち \n\n\u3000\u3000□GAS問合せ（依頼：証明書期限一覧の作成）\n\n\u3000\u3000\u3000\u3000⇒\u3000■04/01) 松ケ野さん、正英さんレビュー\u3000内部向けの資料としては完了\n\n\u3000\u3000\u3000\u3000⇒\u3000GAS展開用の資料作成中\n\n\u3000\u3000□VESPA問合せ（依頼：VESPAの認証方式変更\u3000IDP⇒

In [ ]:
retrieved = retriever.invoke("VESPA お問合せの内容は？")

print(retrieved[0].page_content)

進 捗 状 況 報 告 書（週報）

システム名

																SDR、VDA、MAC



2024年6月11

日

報 告 期 間

2024年6月03日 ～ 2024年6月07

日

作 成 者

石岡雅紀

当



期



作



業



実



施

工程

SDR、VDA、PSPM, DTC Priority

作業項目

SDR、VDA、PSPM, DTC PriorityのAA担当



赤字：待ち

緑字：対応予定

---------------------------------------------------------------------------------------------------

□　業務部署（GAS／XX6等）問合せ／周知　　

　　□05/28) OperationStaffMeeting（6件）

　　　　　⇒■05/31) GAS展開、ADT Nissan team週次会議　

　　　　　※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中

　　　06/04) OperationStaffMeeting（1件）

　　　　　⇒　AMO確認待ち 

　　□GAS問合せ（依頼：証明書期限一覧の作成）

　　　　⇒　■04/01) 松ケ野さん、正英さんレビュー　内部向けの資料としては完了

　　　　⇒　GAS展開用の資料作成中

　　□VESPA問合せ（依頼：VESPAの認証方式変更　IDP⇒OKTA）

　　　　⇒　■04/24) 保守定例内で開発案件となる方向であると確認。またIBXの他部署とも対応方針をそろえた方が良いので、情報収集が必要。

　　　　⇒　■04/25）VESPAよりアンケートフォーム形式で対応状況の聞き取り調査のメール受領

　　　　⇒　■05/09）IB1伊藤-3より、既に1月にOKTAに変更されている旨回答有り

　　　　⇒　AMOにてIB3も対応不要で問題ないか確認中　configの修正対応？

　　　　⇒　保守開発間で返答内容を取りまとめてアンケート回答が必要

　　　　※接続テストの経費が必要になりそう

　　□XX6問合せ（依頼：CDGのレスポンスの遅れにより、CONSULT側でタイムアウト

### 準備：フォーマッターを作る
抽出器の出力を生テキストの形に成形する関数を用意しておく。

In [ ]:
def text_formatter(retriever_output):
  raw_text = retriever_output[0].page_content
  raw_text_wo_newline = raw_text.replace("\n", "")
  return raw_text_wo_newline

In [ ]:
text_formatter(retrieved)

'進 捗 状 況 報 告 書（週報）システム名\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSDR、VDA、MAC2024年6月11日報 告 期 間2024年6月03日 ～ 2024年6月07日作 成 者石岡雅紀当期作業実施工程SDR、VDA、PSPM, DTC Priority作業項目SDR、VDA、PSPM, DTC PriorityのAA担当赤字：待ち緑字：対応予定---------------------------------------------------------------------------------------------------□\u3000業務部署（GAS／XX6等）問合せ／周知\u3000\u3000\u3000\u3000□05/28) OperationStaffMeeting（6件）\u3000\u3000\u3000\u3000\u3000⇒■05/31) GAS展開、ADT Nissan team週次会議\u3000\u3000\u3000\u3000\u3000\u3000※RHEL7.9へアップデート案内にPSPMが対象リストに含まれていなかった理由はAMOで問い合わせ中\u3000\u3000\u300006/04) OperationStaffMeeting（1件）\u3000\u3000\u3000\u3000\u3000⇒\u3000AMO確認待ち \u3000\u3000□GAS問合せ（依頼：証明書期限一覧の作成）\u3000\u3000\u3000\u3000⇒\u3000■04/01) 松ケ野さん、正英さんレビュー\u3000内部向けの資料としては完了\u3000\u3000\u3000\u3000⇒\u3000GAS展開用の資料作成中\u3000\u3000□VESPA問合せ（依頼：VESPAの認証方式変更\u3000IDP⇒OKTA）\u3000\u3000\u3000\u3000⇒\u3000■04/24) 保守定例内で開発案件となる方向であると確認。またIBXの他部署とも対応方針をそろえた方が良いので、情報収集が必要。\u3000\u3000\u3000\u3000⇒\u3000■04/25）VESPAよりアンケートフォーム形式で対応状況の聞き取り調査のメール受領\u30

### 抽出・補完・生成のチェインを作る

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

retriever、prompt_template、llmが用意できたので、これらを組み合わせて、情報の流れを定義する。LangChainでは、これから書いていくような「インプットからアウトプットまでの流れ」をチェイン（chain）と呼ぶ。

In [ ]:

chain = ({"question":RunnablePassthrough(), "context":retriever|text_formatter}
         | prompt_template
         | llm
         )

In [ ]:
chain.invoke("VESPA お問合せの内容は？")

AIMessage(content='VESPAに関する問い合わせの内容は以下の通りです：\n\n1. **VESPAの認証方式変更 (IDP⇒OKTA)**\n   - **04/24**: 保守定例内で開発案件となる方向で確認。また、IBXの他部署とも対応方針をそろえた方が良いので情報収集が必要。\n   - **04/25**: VESPAよりアンケートフォーム形式で対応状況の聞き取り調査のメールを受領。\n   - **05/09**: IB1伊藤-3より、既に1月にOKTAに変更されている旨の回答あり。\n   - **AMO**にてIB3も対応不要で問題ないか確認中（configの修正対応が必要かどうかも含む）。\n   - **保守開発間**で返答内容を取りまとめてアンケート回答が必要。\n   - **接続テストの経費**が必要になりそう。\n\nこれらの内容から、VESPAの認証方式をIDPからOKTAに変更する件についての問い合わせが行われており、それに伴う対応状況の確認や調整が必要であることが記載されています。', response_metadata={'token_usage': {'completion_tokens': 277, 'prompt_tokens': 5891, 'total_tokens': 6168}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-48b06527-8bb7-4b4f-b624-52d7ee0b65b4-0')

In [ ]:
resp=chain.invoke("全体的に問題ポイントを教えて")

In [ ]:
resp.content

'進捗状況報告書の内容から、問題点や改善が必要なポイントを以下に整理します：\n\n### 1. **未解決の課題や対応待ちが多い**\n- **GAS問合せ**：PSPMが対象リストに含まれていなかった理由が未確認（AMO確認待ち）。\n- **VESPA問合せ**：アンケートの返答が未完了（AMO確認待ち）。\n- **XX6問合せ**：CPU高騰の原因調査が未完了（AMO確認待ち）。\n- **SDRでのH15 Key登録エラー**：Daimlerからの返信待ち。\n- **TLS1.0無効化対応**：6/10の動確完了連絡待ち。\n- **BCVデータ重複対応**：恒久対応後の進め方が未確定（業連の調整中）。\n\n### 2. **連絡や確認待ちが多い**\n- **AMO確認待ち**：複数の項目でAMOからの確認や回答待ちが多い。\n- **富士通AMOからの連絡待ち**：SDRログ容量増加対応、PSPM Cookie影響調査、BCVデータ重複対応など。\n\n### 3. **調整や確認に時間がかかっている**\n- **業連調整**：業連の発行や調整が複数あり、進捗が遅れている。\n- **VESPAリプロチェック**：リリース後の動確がまだ完了していない。\n\n### 4. **明確なスケジュールや期限が不明**\n- 各タスクや対応の完了予定日が明確に示されていないため、進捗管理が難しい可能性がある。\n\n### 5. **重複したタスクや対応の優先順位が不明確**\n- 複数の重複したタスクが存在し、どのタスクが優先されるべきかが明確ではない。\n\n### 6. **コミュニケーション不足**\n- 幾つかの項目で連絡が遅れていたり、返答がないまま時間が経過している。\n- 例：監視アラートの連絡先変更が未完了。\n\n### 7. **費用の扱いや予算調整が不明確**\n- 費用の扱いや予算調整に関して、具体的な計画や進捗が見られない。\n\n### 8. **障害対応の再発防止策が不明確**\n- 障害の原因調査や再発防止策についての具体的な対応策が明示されていない。\n\n### 改善の提案\n1. **進捗の可視化**：未解決の課題や対応待ちの項目を一覧化し、進捗状況を定期的に更新・共有する。\n2. **期限設定**：各タスクや

In [ ]:
resp=chain.invoke("VESPAリプロチェック　リリース後の動作確認は完了してますか？")

In [ ]:
resp.content

'「VESPAリプロチェック\u3000リリース後の動作確認未完了」という記述は、システムやソフトウェアの変更や更新が行われた後、その変更が正しく機能するかどうかを確認するためのテスト（動作確認）がまだ完了していないことを意味します。具体的には以下のような状況が考えられます：\n\n1. **リリース後のテストが未完了**：ソフトウェアの新しいバージョンや修正が本番環境にリリースされた後、その動作が期待通りであるかどうかの検証がまだ行われていない。\n\n2. **確認作業の遅延**：動作確認を担当するチームや個人が、他のタスクやスケジュールの都合でまだ確認作業を完了していない。\n\n3. **問題発見の可能性**：動作確認が完了していないため、リリースされた変更に何らかの問題や不具合が潜在的に含まれている可能性がある。\n\nこのため、「リリース後の動作確認未完了」とは、リリースされたソフトウェア変更が安定して動作することを確認するプロセスがまだ完了していないため、潜在的なリスクが残っている状態を示しています。\n\n具体的な進捗状況としては、2024年6月10日に「リリース後の動確完了」となっているため、その時点で動作確認が完了していることが報告されています。したがって、現時点では動作確認が完了し、問題がないことが確認されたことになります。'

In [ ]:
resp=chain.invoke("全体的に問題ポイントに「VESPAリプロチェック　リリース後の動作確認未完了」はどいう意味でしょうか？")

In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='Sdr', type='System'), Node(id='Vda', type='System'), Node(id='Mac', type='System'), Node(id='石岡雅紀', type='Person')], relationships=[Relationship(source=Node(id='Sdr', type='System'), target=Node(id='石岡雅紀', type='Person'), type='CONTRIBUTOR'), Relationship(source=Node(id='Vda', type='System'), target=Node(id='石岡雅紀', type='Person'), type='CONTRIBUTOR'), Relationship(source=Node(id='Mac', type='System'), target=Node(id='石岡雅紀', type='Person'), type='CONTRIBUTOR')], source=Document(page_content='進 捗 状 況 報 告 書（週報）\n\nシステム名\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tSDR、VDA、MAC\n\n\n\n2024年6月3\n\n日\n\n報 告 期 間\n\n2024年5月27日 ～ 2024年5月31\n\n日\n\n作 成 者\n\n石岡雅紀\n\n当\n\n\n\n期\n\n\n\n作\n\n\n\n業\n\n\n\n実\n\n\n\n施\n\n工程\n\nSDR、VDA、PSPM, DTC Priority\n\n作業項目\n\nSDR、VDA、PSPM, DTC PriorityのAA担当\n\n\n\n赤字：待ち\n\n緑字：対応予定\n\n---------------------------------------------------------------------------------------------------\n\n□\u3000業務部署（GAS／XX6等）問合せ／周知\u3000\u3000\n\n\u3000\u3000■05/2